## Logic for lambda cron task

In [ ]:
'''Getting batch data'''
import requests
import os

crypto_api_key = os.environ['CRYPTO_API_KEY']
headers = {
    'authorization': 'Apikey ' + crypto_api_key
}

response = requests.get('https://min-api.cryptocompare.com/data/v2/histohour?fsym=BTC&tsym=USD&limit=30')

In [ ]:
'''Getting from api response'''
if response.status_code == 200:
    data = response.json()['Data']['Data']
else:
    print('There is an error in the response')

In [ ]:
data

In [ ]:
import datetime

datetime.datetime.fromtimestamp(data[len(data)-1]['time'])

In [ ]:
'''Creating DataFrame from response data'''
import pandas as pd

df = pd.DataFrame(data)
df = df.drop(['conversionSymbol', 'conversionType'], axis=1)

In [ ]:
'''Calculating simple moving average from batch'''
import numpy as np

def calculate_moving_average(av_type, df):
    values = df['high'].values
    values = np.flip(values)
    if av_type == 'short':
        moving_average = sum(values[0:9])/len(values[0:9])
        print(len(values[0:9]))
    elif av_type == 'long':
        print(len(values[0:30]))
        moving_average = sum(values[0:30])/len(values[0:30])
    return moving_average

short_moving_average = calculate_moving_average(av_type='short', df=df)
long_moving_average = calculate_moving_average(av_type='long', df=df)
print(short_moving_average)
print(long_moving_average)

In [ ]:
24*30

## Experiment with 2000 hours

In [ ]:
'''Making request to get bunch data'''
import requests

crypto_api_key = os.environ['CRYPTO_API_KEY']
headers = {
    'authorization': 'Apikey ' + crypto_api_key
}

response = requests.get('https://min-api.cryptocompare.com/data/v2/histohour?fsym=BTC&tsym=USD&limit=2000')

In [ ]:
'''Getting data from api response'''
if response.status_code == 200:
    data = response.json()['Data']['Data']
else:
    print('There is an error in the response')

In [ ]:
''' Saving data for testing purposes'''
import pickle
from datetime import datetime
import numpy as np

data_array = np.array([])
for i in data:
    data_array = np.append(data_array, i['high'])

file = open('btc_prices.pk', 'wb')
pickle.dump(data_array, file)
file.close()

In [ ]:
import time

class TradingSimulator:
    def __init__(self, initial_amount, historical_prices):
        self.current_currency = 'USD'
        self.moving_averages = {'last': {'long':None, 'short':None}, 'now': {'long':None, 'short':None}}
        self.initial_amount = initial_amount
        self.current_amount = initial_amount
        self.last_buying_amount = initial_amount
        self.cumulative_earnings = 0
        self.historical_prices = historical_prices
        if len(historical_prices) > 21:
            self.length = len(historical_prices)
        else:
            raise Exception('Simulator needs at least 21 data points')
    
    def start(self):
        self.moving_averages['last']['long'] = self.calculate_moving_average(av_type='long')
        self.moving_averages['last']['short'] = self.calculate_moving_average(av_type='short')
        
        for i, item in enumerate(self.historical_prices[21:], 21):
            #time.sleep(1)
            self.moving_averages['now']['long'] = self.calculate_moving_average(av_type='long', index=i)
            self.moving_averages['now']['short'] = self.calculate_moving_average(av_type='short', index=i)
            self.decide_action(price = item)
            self.moving_averages['last']['long'] = self.moving_averages['now']['long']
            self.moving_averages['last']['short'] = self.moving_averages['now']['short']
        return self.current_usd()
    
    def calculate_moving_average(self, av_type, index=21):
        short_values = self.historical_prices[index-9:index]
        long_values = self.historical_prices[index-21:index]
        if av_type == 'short':
            moving_average = sum(short_values)/len(short_values)
        elif av_type == 'long':
            moving_average = sum(long_values)/len(long_values)
        return moving_average
    
    def decide_action(self, price):
        if self.current_currency == 'BTC':
            #Selling with 10% earnings rule
            if self.current_amount * price > 1.05 * self.last_buying_amount:
                if self.current_amount * price > self.initial_amount:
                    self.cumulative_earnings = self.cumulative_earnings + (self.current_amount * price) - self.initial_amount
                    self.current_amount = self.initial_amount
                else:
                    self.current_amount = self.current_amount * price
                self.current_currency = 'USD'
                print('Earnings taken: SELL', self.current_amount, self.current_currency)

            #STOP LOSS
            '''
            elif self.current_amount * price < 0.9 * self.last_buying_amount:
                self.current_amount = self.current_amount * price
                self.current_currency = 'USD'
                print('Stop Loss: SELL', self.current_amount, self.current_currency)
            '''
        #Bull tendency case
        if self.moving_averages['now']['long'] > self.moving_averages['now']['short']:
            if self.moving_averages['last']['long'] < self.moving_averages['last']['short']:
                if self.current_currency == 'USD':
                    self.current_amount = self.current_amount / price
                    self.current_currency = 'BTC'
                    print('Short cross up: BUY', self.current_amount, self.current_currency)
            else:
                print('Tendency is the same as last hour')
        #Bear tendency case
        else:
            if self.moving_averages['last']['long'] > self.moving_averages['last']['short']:
                if self.current_currency == 'BTC':
                    if self.current_amount * price > self.initial_amount:
                        self.cumulative_earnings = self.cumulative_earnings + (self.current_amount * price) - self.initial_amount
                        self.current_amount = self.initial_amount
                    else:
                        self.current_amount = self.current_amount * price
                    self.current_currency = 'USD'
                    print('Short cross down: SELL', self.current_amount, self.current_currency)
            else:
                print('Tendency is the same as last hour')
    
    def current_usd(self):
        if self.current_currency == 'USD':
            return self.current_amount + self.cumulative_earnings
        else:
            return (self.current_amount * self.historical_prices[len(self.historical_prices) - 1]) + self.cumulative_earnings

In [ ]:
simulator = TradingSimulator(initial_amount = 100 , historical_prices = data_array)

In [ ]:
result = simulator.start()

In [ ]:
print(result)

# Bitso

In [ ]:
import bitso
import os

key = os.environ['BITSO_API_KEY']
secret = os.environ['BITSO_API_SECRET']

api = bitso.Api(key, secret)

In [ ]:
status = api.account_status()
status.daily_limit

In [ ]:
trades = api.trades('btc_mxn')
last_sell = None
last_buy = None
for trade in trades:
    if not last_sell:
        if trade.maker_side == 'sell':
            last_sell = trade
    if not last_buy:
        if trade.maker_side == 'buy':
            last_buy = trade
    if last_sell and last_buy:
        break
last_buy

In [ ]:
last_sell

In [ ]:
len(trades)

In [ ]:
fees = api.fees()

In [ ]:
fees.btc_mxn.fee_percent

In [ ]:
from decimal import Decimal
x = Decimal(str(100.92382983))
x